# Containerized Deployment with Docker

Running NVIDIA FLARE in a Docker container is sometimes a convenient way to ensure a uniform OS and software environment across client and server systems. This can be used as an alternative to the bare-metal Python virtual environment described above and will use a similar installation to simplify transitioning between a bare metal and containerized environment.

To get started with a containerized deployment, you will first need to install a supported container runtime and the NVIDIA Container Toolkit to enable support for GPUs. System requirements and instructions for this can be found in the NVIDIA Container Toolkit Install Guide <https://docs.nvidia.com/datacenter/cloud-native/container-toolkit/install-guide.html>.

A simple Dockerfile is used to capture the base requirements and dependencies. In this case, we’re building an environment that will support PyTorch-based workflows, in particular the Hello PyTorch with Tensorboard Streaming example. The base for this build is the NGC PyTorch container. On this base image, we will install the necessary dependencies and clone the NVIDIA FLARE GitHub source code into the root workspace directory. To create a Dockerfile, create a file named Dockerfile using any text editor and include the following:

```
ARG PYTORCH_IMAGE=nvcr.io/nvidia/pytorch:23.02-py3
FROM ${PYTORCH_IMAGE}

RUN python3 -m pip install -U pip
RUN python3 -m pip install -U setuptools
RUN python3 -m pip install torch torchvision tensorboard nvflare

WORKDIR /workspace/
RUN git clone https://github.com/NVIDIA/NVFlare.git

```


We can then build the new container by running docker build in the directory containing this Dockerfile, for example tagging it nvflare-pt:

docker build -t nvflare-pt .
This will result in a docker image, nvflare-pt:latest. You can run this container with Docker, in this example mounting a local my-workspace directory into the container for use as a persistent workspace:

```
mkdir my-workspace
docker run --rm -it --gpus all \
    --ipc=host --ulimit memlock=-1 --ulimit stack=67108864 \
    -w $(pwd -P)/my-workspace:/workspace/my-workspace \
    nvflare-pt:latest
```    
Once the container is running, you can also exec into the container, for example if you need another terminal to start additional FLARE clients. First find the CONTAINER ID using docker ps, and then use that ID to exec into the container:
```
docker ps  # use the CONTAINER ID in the output
docker exec -it <CONTAINER ID> /bin/bash
```
This container can be used to run the FL Simulator or any FL server or client. When using the FL Simulator (described in the next section), you can simply mount in any directories needed for your FLARE application code, and run the Simulator within the Docker container with all dependencies installed.